---
title: "Assignment"
author: "Marcin Zinówko"
date: today
format: html
execute:
  echo: false      
---

In [ ]:
season_no = 1  # Use Quarto's parameter syntax to define season number

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import date

# Define the URL for the Wikipedia page of the specific Game of Thrones season
wikipedia_url = f"https://en.wikipedia.org/wiki/Game_of_Thrones_(season_{season_no})"

# Fetch data from Wikipedia
response = requests.get(wikipedia_url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from Wikipedia for season {season_no}. Please check the URL.")

# Use pandas to parse the HTML and extract tables
tables = pd.read_html(response.text)

# Extract the correct table containing the episode data
# The episode data is usually the first table with more than 2 columns and contains 'Title' in the columns
season_data = None
for table in tables:
    if len(table.columns) > 2 and 'Title' in table.columns:
        season_data = table
        break

if season_data is None:
    raise Exception(f"Could not find episode data table for season {season_no} on Wikipedia.")

# Clean up the data frame
season_data.columns = [col.lower().replace(' ', '_') for col in season_data.columns]  # Normalize column names
season_data['viewers'] = pd.to_numeric(season_data['us_viewers(millions)'], errors='coerce')  # Convert viewers to numeric

# Calculate summary statistics
episodes_no = len(season_data)
date_start = season_data['original_air_date'].iloc[0]
date_end = season_data['original_air_date'].iloc[-1]

viewers = season_data['viewers'].mean()

viewers_start = season_data['viewers'].iloc[0]
viewers_end = season_data['viewers'].iloc[-1]

max_viewers = season_data['viewers'].max()
top_episode_desc = season_data.loc[season_data['viewers'] == max_viewers, 'title'].values[0]

# Game of Thrones - Season Summary in Numbers


In [ ]:
print(f"Season {season_no} of Game of Thrones consisted of {episodes_no} episodes that aired between {date_start} and {date_end} on HBO.")
print(f"The show gathered an average of {viewers:.2f} million first-day TV viewers in the US, with the number growing from {viewers_start:.2f} to {viewers_end:.2f} million by the end of the season.")
print(f'The most popular episode of the season was "{top_episode_desc}".')

# Viewership Data


In [ ]:
plt.plot(season_data['viewers'], color='red', linewidth=5)
plt.xlabel("Episode number")
plt.ylabel("1st day TV viewers in the US (millions)")
plt.title(f"Game of Thrones Season {season_no} Viewership")
plt.show()

# Episodes with Above-Average Viewership


In [ ]:
above_average_episodes = season_data[season_data['viewers'] > viewers][['no.', 'title', 'directed_by']]
print("Episodes with above-average viewership:")
print(above_average_episodes)